# Decision Trees for regression tasks

In [ ]:
# import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate,\
                                    cross_val_score, train_test_split, ShuffleSplit, validation_curve
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn import tree


In [ ]:
np.random.seed(306)

In [ ]:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

## Load the dataset

In [ ]:
features, labels = fetch_california_housing(as_frame=True, return_X_y=True)

#train - test split
com_train_features, test_features, com_train_labels,  test_labels = train_test_split(features, labels, random_state=42)

# train ---> train + dev split
train_features, dev_features, train_labels, dev_labels = train_test_split(com_train_features, com_train_labels, random_state=42)

## Model Set up

In [ ]:
from numpy import std


dt_reg_pipeline = Pipeline([("scaling", StandardScaler()),
                            ("dt_reg", DecisionTreeRegressor(max_depth=3, random_state=42))])

dt_reg_cv_results = cross_validate(dt_reg_pipeline,
                                    com_train_features,
                                    com_train_labels,
                                    cv = cv,
                                    scoring = "neg_mean_absolute_error",
                                    return_train_score=True,
                                    return_estimator=True)

dt_reg_train_error = -1 * dt_reg_cv_results['train_score']
dt_reg_test_error = -1 * dt_reg_cv_results['test_score']

print(f"Mean absolute error on the training set: \n"
        f"{dt_reg_train_error.mean():.3f} +/- {dt_reg_train_error.std():.3f}")
print(f"Mean absolute error on the testing set: \n"
        f"{dt_reg_test_error.mean():.3f} +/- {dt_reg_test_error.std():.3f}")

## Visualising the tree

### As a tree diagram

In [ ]:
dt_reg_pipeline.fit(train_features, train_labels)

In [ ]:
plt.figure(figsize=(28,8), facecolor='w')

a = tree.plot_tree(dt_reg_pipeline[-1],
                    feature_names = features.columns,
                    rounded = True,
                    filled = True,
                    fontsize = 12)

plt.show()

### As a text based diagram

In [ ]:
tree_rules = export_text(dt_reg_pipeline[-1])
print(tree_rules)

## Evaluation

In [ ]:
test_labels_pred = dt_reg_pipeline.predict(test_features)

mae = mean_absolute_error(test_labels, test_labels_pred)
mse = mean_squared_error(test_labels, test_labels_pred)

print("MAE: ", mae)
print("MSE: ", mse)

## HP tuning

In [ ]:
param_grid = {'dt_reg__max_depth': range(1, 20),
            'dt_reg__min_samples_split': range(2, 8)}
dt_grid_search = GridSearchCV(dt_reg_pipeline,
                                param_grid=param_grid,
                                n_jobs=2,
                                cv= cv,
                                scoring='neg_mean_absolute_error',
                                return_train_score=True)
dt_grid_search.fit(com_train_features, com_train_labels)

In [ ]:
dt_grid_search.best_params_

In [ ]:
dt_reg_pipeline.set_params(dt_reg__max_depth=11, dt_reg__min_samples_split=5).fit(com_train_features, com_train_labels)

In [ ]:
test_labels_pred = dt_reg_pipeline.predict(test_features)

mae = mean_absolute_error(test_labels, test_labels_pred)
mse = mean_squared_error(test_labels, test_labels_pred)
r2 = r2_score(test_labels, test_labels_pred)

print("MAE: ", mae)
print("MSE: ", mse)
print("R2 score: ", r2)